In [1]:
import re


def get_top_n_users_by_accuracy(filename, n, min_questions):
    with open(filename, 'r', encoding='utf-8') as file:
        data = file.read()
    
    user_pattern = re.findall(r'\[(U\d+)] : 평균 nDCG@5 : ([\d.]+)  \|  평균 nDCG@10 : ([\d.]+)  \|  Accuracy : ([\d.]+) \((\d+)/(\d+)\)', data)
    
    users = []
    for user in user_pattern:
        user_id, ndcg5, ndcg10, accuracy, correct, total = user
        total = int(total)
        if total >= min_questions:
            accuracy = float(accuracy)
            users.append((user_id, float(ndcg5), float(ndcg10), accuracy, int(correct), total))
    
    top_users = sorted(users, key=lambda x: x[3], reverse=True)[:n]
    
    for i, (user_id, ndcg5, ndcg10, accuracy, correct, total) in enumerate(top_users, 1):
        print(f"{i}. {user_id} [ 평균 nDCG@5 : {ndcg5:.3f}  |  평균 nDCG@10 : {ndcg10:.3f}  |  Accuracy : {accuracy:.3f} ({correct}/{total}) ]")



In [18]:
# 예제 실행
file_name = '[250220] negative_acc2_metrics.txt'
file_name = f'../../results/metrics/{file_name}'
get_top_n_users_by_accuracy(file_name, n=10, min_questions=10)

1. U53 [ 평균 nDCG@5 : 0.400  |  평균 nDCG@10 : 0.400  |  Accuracy : 0.400 (6/15) ]
2. U525 [ 평균 nDCG@5 : 0.400  |  평균 nDCG@10 : 0.400  |  Accuracy : 0.400 (4/10) ]
3. U138 [ 평균 nDCG@5 : 0.364  |  평균 nDCG@10 : 0.364  |  Accuracy : 0.364 (4/11) ]
4. U607 [ 평균 nDCG@5 : 0.364  |  평균 nDCG@10 : 0.364  |  Accuracy : 0.364 (4/11) ]
5. U839 [ 평균 nDCG@5 : 0.364  |  평균 nDCG@10 : 0.364  |  Accuracy : 0.364 (4/11) ]
6. U745 [ 평균 nDCG@5 : 0.353  |  평균 nDCG@10 : 0.353  |  Accuracy : 0.353 (6/17) ]
7. U564 [ 평균 nDCG@5 : 0.350  |  평균 nDCG@10 : 0.350  |  Accuracy : 0.350 (7/20) ]
8. U725 [ 평균 nDCG@5 : 0.350  |  평균 nDCG@10 : 0.350  |  Accuracy : 0.350 (7/20) ]
9. U246 [ 평균 nDCG@5 : 0.333  |  평균 nDCG@10 : 0.333  |  Accuracy : 0.333 (5/15) ]
10. U356 [ 평균 nDCG@5 : 0.333  |  평균 nDCG@10 : 0.333  |  Accuracy : 0.333 (4/12) ]


In [2]:
file_name = '[fine2] negative_metrics.txt'
file_name = f'../../results/metrics/{file_name}'
get_top_n_users_by_accuracy(file_name, n=10, min_questions=5)

1. U15197 [ 평균 nDCG@5 : 0.800  |  평균 nDCG@10 : 0.800  |  Accuracy : 0.800 (4/5) ]
2. U15093 [ 평균 nDCG@5 : 0.714  |  평균 nDCG@10 : 0.714  |  Accuracy : 0.714 (5/7) ]
3. U15473 [ 평균 nDCG@5 : 0.667  |  평균 nDCG@10 : 0.667  |  Accuracy : 0.667 (4/6) ]
4. U15142 [ 평균 nDCG@5 : 0.600  |  평균 nDCG@10 : 0.600  |  Accuracy : 0.600 (3/5) ]
5. U15098 [ 평균 nDCG@5 : 0.500  |  평균 nDCG@10 : 0.500  |  Accuracy : 0.500 (3/6) ]
6. U15083 [ 평균 nDCG@5 : 0.455  |  평균 nDCG@10 : 0.455  |  Accuracy : 0.455 (5/11) ]
7. U15426 [ 평균 nDCG@5 : 0.429  |  평균 nDCG@10 : 0.429  |  Accuracy : 0.429 (3/7) ]
8. U15056 [ 평균 nDCG@5 : 0.400  |  평균 nDCG@10 : 0.400  |  Accuracy : 0.400 (2/5) ]
9. U15185 [ 평균 nDCG@5 : 0.400  |  평균 nDCG@10 : 0.400  |  Accuracy : 0.400 (2/5) ]
10. U15227 [ 평균 nDCG@5 : 0.400  |  평균 nDCG@10 : 0.400  |  Accuracy : 0.400 (2/5) ]


In [3]:
def parse_txt(file_path):
    """Parse the given txt file and return a dictionary of user accuracy and correct questions."""
    user_data = {}
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.readlines()
    
    user_id = None
    for line in content:
        user_match = re.match(r'\[(U\d+)]', line)
        if user_match:
            user_id = user_match.group(1)
            acc_match = re.search(r'Accuracy : ([0-9\.]+) \((\d+)/(\d+)\)', line)
            if acc_match:
                accuracy = float(acc_match.group(1))
                correct_answers = set()
                user_data[user_id] = {'accuracy': accuracy, 'correct_answers': correct_answers}
        
        question_match = re.match(r'\s*- Question (\d+) : nDCG@5 = ([0-9\.]+)', line)
        if question_match and user_id:
            question_num = int(question_match.group(1))
            ndcg_5 = float(question_match.group(2))
            if ndcg_5 == 1.0:
                user_data[user_id]['correct_answers'].add(question_num)
    
    return user_data

def compare_users(abc_path, qwe_path):
    """Compare users based on Accuracy from two files and count various categories."""
    abc_users = parse_txt(abc_path)
    qwe_users = parse_txt(qwe_path)
    
    higher, equal, lower = 0, 0, 0
    same_correct_questions = 0
    
    for user in qwe_users:
        if user in abc_users:
            if qwe_users[user]['accuracy'] > abc_users[user]['accuracy']:
                higher += 1
            elif qwe_users[user]['accuracy'] < abc_users[user]['accuracy']:
                lower += 1
            else:
                equal += 1
                if qwe_users[user]['correct_answers'] == abc_users[user]['correct_answers']:
                    same_correct_questions += 1
    
    return {
        'higher_accuracy': higher,
        'equal_accuracy': equal,
        'lower_accuracy': lower,
        'same_correct_questions': same_correct_questions
    }

In [6]:
# Example usage
file1 = '[250220] negative_ndcg2_metrics.txt'
file1 = f'../../results/metrics/{file1}'

file2 = '[250220] negative_acc2_metrics.txt'
file2 = f'../../results/metrics/{file2}'

result = compare_users(file1, file2)
result

{'higher_accuracy': 319,
 'equal_accuracy': 442,
 'lower_accuracy': 239,
 'same_correct_questions': 331}